In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [3]:
from dotenv import load_dotenv
import nest_asyncio
import os

os.environ["SWE_AGENT_CONFIG_DIR"] = "."
os.environ["SWE_AGENT_TOOLS_DIR"] = "tools"
os.environ["SWE_AGENT_TRAJECTORY_DIR"] = "trajectories"

os.makedirs("replays", exist_ok=True)
os.makedirs("trajectories", exist_ok=True)

load_dotenv()
nest_asyncio.apply()

In [4]:
import art
from art.local import LocalBackend
from rollout import ModelConfig

backend = LocalBackend()
model = art.TrainableModel(
    name="001",
    project="sweagent",
    config=ModelConfig(
        max_input_tokens=32_768,
        system_prompt_suffix="\n/no_think",
        xml_function_calling=True,
    ),
    base_model="Qwen/Qwen3-32B",
)
await model.register(backend)

INFO 05-31 21:06:16 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-31 21:06:16 [__init__.py:239] Automatically detected platform cuda.


/home/sky/sky_workdir/src/art/__init__.py:11: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth  # type: ignore


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-31 21:06:25 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-31 21:06:25 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.5.1: Fast Qwen3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA H200. Num GPUs = 1. Max memory: 139.719 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen3-32b-bnb-4bit with actual GPU utilization = 78.66%
Unsloth: Your GPU has CUDA compute capability 9.0 with VRAM = 139.72 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill toke

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.33it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.31it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.40it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.30it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.32it/s]

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.25it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.26it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.36it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.32it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.31i

INFO 05-31 21:06:41 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 05-31 21:06:42 [model_runner.py:1140] Model loading took 18.1555 GiB and 6.659566 seconds
INFO 05-31 21:06:48 [worker.py:287] Memory profiling takes 5.83 seconds
INFO 05-31 21:06:48 [worker.py:287] the current vLLM instance can use total_gpu_memory (139.72GiB) x gpu_memory_utilization (0.79) = 109.90GiB
INFO 05-31 21:06:48 [worker.py:287] model weights take 18.16GiB; non_torch_memory takes 0.15GiB; PyTorch activation peak memory takes 6.53GiB; the rest of the memory reserved for KV Cache is 85.07GiB.
INFO 05-31 21:06:48 [executor_base.py:112] # cuda blocks: 21777, # CPU blocks: 1536
INFO 05-31 21:06:48 [executor_base.py:117] Maximum concurrency for 32768 tokens per request: 10.63x
INFO 05-31 21:06:50 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the C

Capturing CUDA graph shapes: 100%|█████████████| 53/53 [00:51<00:00,  1.03it/s]


INFO 05-31 21:07:42 [model_runner.py:1592] Graph capturing finished in 52 secs, took 3.52 GiB
INFO 05-31 21:07:42 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 60.66 seconds
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm']
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm']


Unsloth 2025.5.1 patched 64 layers with 64 QKV layers, 64 O layers and 64 MLP layers.
Unsloth: Already have LoRA adapters! We shall skip this step.


In [5]:
import asyncio
import modal

sandboxes: list[modal.Sandbox] = []
async for sandbox in modal.Sandbox.list.aio(
    app_id=modal.App.lookup("swe-rex", create_if_missing=True).app_id
):
    sandboxes.append(sandbox)
_ = await asyncio.gather(*[sandbox.terminate.aio() for sandbox in sandboxes])

In [6]:
from instances import as_instances_iter, get_filtered_swe_smith_instances_df
from rollout import rollout

tasks = (
    get_filtered_swe_smith_instances_df()
    .sample(fraction=1.0, shuffle=True, seed=42)
    .pipe(as_instances_iter)
)

async for batch in art.trajectory_group_batches(
    (art.TrajectoryGroup(rollout(model, task) for _ in range(4)) for task in tasks),
    batch_size=8,
    max_batch_exceptions=128,
    max_concurrent_batches=4,
    skip_batches=await model.get_step(),
):
    await model.train(batch)

batches:   0%|          | 0/32 [00:00<?, ?it/s]

CRITICAL:rex-deploy-asyncio_0:Traceback: 
  + Exception Group Traceback (most recent call last):
  |   File "/root/.local/pipx/.cache/eac1061a7f38818/lib/python3.10/site-packages/starlette/_utils.py", line 76, in collapse_excgroups
  |     yield
  |   File "/root/.local/pipx/.cache/eac1061a7f38818/lib/python3.10/site-packages/starlette/middleware/base.py", line 177, in __call__
  |     async with anyio.create_task_group() as task_group:
  |   File "/root/.local/pipx/.cache/eac1061a7f38818/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 772, in __aexit__
  |     raise BaseExceptionGroup(
  | exceptiongroup.ExceptionGroup: unhandled errors in a TaskGroup (1 sub-exception)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "/root/.local/pipx/.cache/eac1061a7f38818/lib/python3.10/site-packages/starlette/middleware/errors.py", line 165, in __call__
    |     await self.app(scope, receive, _send)
    |   File "/root/.local/pipx/.cac

swerex.exceptions SessionDoesNotExistError


CRITICAL:rex-deploy-asyncio_0:Traceback: 
  + Exception Group Traceback (most recent call last):
  |   File "/root/.local/pipx/.cache/eac1061a7f38818/lib/python3.10/site-packages/starlette/_utils.py", line 76, in collapse_excgroups
  |     yield
  |   File "/root/.local/pipx/.cache/eac1061a7f38818/lib/python3.10/site-packages/starlette/middleware/base.py", line 177, in __call__
  |     async with anyio.create_task_group() as task_group:
  |   File "/root/.local/pipx/.cache/eac1061a7f38818/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 772, in __aexit__
  |     raise BaseExceptionGroup(
  | exceptiongroup.ExceptionGroup: unhandled errors in a TaskGroup (1 sub-exception)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "/root/.local/pipx/.cache/eac1061a7f38818/lib/python3.10/site-packages/starlette/middleware/errors.py", line 165, in __call__
    |     await self.app(scope, receive, _send)
    |   File "/root/.local/pipx/.cac

swerex.exceptions SessionDoesNotExistError


...

swerex.exceptions SessionDoesNotExistError


CRITICAL:rex-deploy-asyncio_18:Traceback: 
  + Exception Group Traceback (most recent call last):
  |   File "/root/.local/pipx/.cache/eac1061a7f38818/lib/python3.10/site-packages/starlette/_utils.py", line 76, in collapse_excgroups
  |     yield
  |   File "/root/.local/pipx/.cache/eac1061a7f38818/lib/python3.10/site-packages/starlette/middleware/base.py", line 177, in __call__
  |     async with anyio.create_task_group() as task_group:
  |   File "/root/.local/pipx/.cache/eac1061a7f38818/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 772, in __aexit__
  |     raise BaseExceptionGroup(
  | exceptiongroup.ExceptionGroup: unhandled errors in a TaskGroup (1 sub-exception)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "/root/.local/pipx/.cache/eac1061a7f38818/lib/python3.10/site-packages/starlette/middleware/errors.py", line 165, in __call__
    |     await self.app(scope, receive, _send)
    |   File "/root/.local/pipx/.ca

swerex.exceptions SessionDoesNotExistError


CRITICAL:rex-deploy-asyncio_18:Traceback: 
  + Exception Group Traceback (most recent call last):
  |   File "/root/.local/pipx/.cache/eac1061a7f38818/lib/python3.10/site-packages/starlette/_utils.py", line 76, in collapse_excgroups
  |     yield
  |   File "/root/.local/pipx/.cache/eac1061a7f38818/lib/python3.10/site-packages/starlette/middleware/base.py", line 177, in __call__
  |     async with anyio.create_task_group() as task_group:
  |   File "/root/.local/pipx/.cache/eac1061a7f38818/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 772, in __aexit__
  |     raise BaseExceptionGroup(
  | exceptiongroup.ExceptionGroup: unhandled errors in a TaskGroup (1 sub-exception)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "/root/.local/pipx/.cache/eac1061a7f38818/lib/python3.10/site-packages/starlette/middleware/errors.py", line 165, in __call__
    |     await self.app(scope, receive, _send)
    |   File "/root/.local/pipx/.ca

swerex.exceptions SessionDoesNotExistError
